### 1) Load necessary Python packages

In [2]:
import os
import sys
import dill

# Add the local src directory to the path
sys.path.append('./src/')

# Load the GBA_model class
from GBA_model import *

### 2) Define a function to load a model from a binary backup
See the script `csv_to_binary.py`.

In [3]:
### Load the model from a binary file ###
def load_model( model_name ):
    filename = "./binary_models/"+model_name+".gba"
    assert os.path.isfile(filename), "ERROR: model not found."
    ifile = open(filename, "rb")
    model = dill.load(ifile)
    ifile.close()
    return model

### 3) Load the model "EC12b" (a very abstract model of <em>E. coli</em>)

In [4]:
# I previously saved the model as a binary file with the script `csv_to_binary.py`
model = load_model("EC12b")

### 4) Define the condition
The condition defines the environmental state of the cell (the composition of the medium). External concentrations are stored in the vector $x$.
Conditions are listed for the EC12b model in `./csv_models/EC12b/conditions.csv`.
We will use the condition 1.

In [5]:
model.set_condition("1")

### 5) Calculate the initial state
This little linear program routine allows us to find at least one suboptimal solution for the GBA model.
The solution (like every solutions) must satisfy the following constraints:
- $c > 0$ (positivity of mass concentrations)
- $p > 0$ (positivity of protein mass concentrations)
- $sM . f = 1$ (Relative density of 1)

The routine requires GUROBIpy to be installed.

In [6]:
model.solve_local_linear_problem()

### 6) Calculate all model variables

In [7]:
model.calculate()

### 7) Display a few variables

In [22]:
print("\n> Summary of the model:")
print(model)


> Summary of the model:

 -------- Model report: EC12b --------
| • Nb metabolites          = 16
| • Nb external metabolites = 4
| • Nb internal metabolites = 12
 -------------------------------------
| • Nb reactions          = 12
| • Nb exchange reactions = 3
| • Nb internal reactions = 8
 -------------------------------------



In [19]:
print("\n> Flux fractions f:")
df = pd.DataFrame(data=model.f, index=model.reaction_ids, columns=["f"])
df


> Flux fractions f:


,f
tC,0.558586
tP,0.220707
tN,0.220707
tAce,0.000000
Resp,0.261373
Ferm,0.041667
Nucprod,0.683019
Aaprod,0.227330
Replication,0.008333
Transcription,0.598050


In [20]:
print("\n> Fluxes v:")
df = pd.DataFrame(data=model.v, index=model.reaction_ids, columns=["v"])
df


> Fluxes v:


,f
tC,25.247762
tP,9.975826
tN,9.975826
tAce,0.000000
Resp,11.813886
Ferm,1.883309
Nucprod,30.872042
Aaprod,10.275180
Replication,0.376662
Transcription,27.031514


In [21]:
print("\n> Mass fractions b:")
df = pd.DataFrame(data=model.b, index=model.c_ids, columns=["b"])
df


> Mass fractions b:


,b
C,0.008333
P,0.008333
N,0.008333
Ace,0.008333
ADP,0.139020
ATP,0.008333
AA,0.008333
Nuc,0.008333
DNA,0.008333
tRNA,0.008333


In [23]:
print("\n> Mass concentrations c:")
df = pd.DataFrame(data=model.c, index=model.c_ids, columns=["c"])
df


> Mass concentrations c:


,c
C,2.833333
P,2.833333
N,2.833333
Ace,2.833333
ADP,47.266667
ATP,2.833333
AA,2.833333
Nuc,2.833333
DNA,2.833333
tRNA,2.833333


In [24]:
print("\n> Growth rate mu:")
print(model.mu)


> Growth rate mu:
0.13293945281963943


### 8) Test the consistency of the model (positivity + total density)
The result of the consistency check is stored in the variable model.consistent

In [27]:
model.check_model_consistency()
print(model.consistent)

True


### 9) Mu derivatives
The function `model.calculate()` also calculates derivatives $\dfrac{d\mu}{df}$.
To follow the fitness gradient, the important variable is `GCC_f`, which corrects for total density.

In [28]:
print("\n> dmu/df:")
df = pd.DataFrame(data=model.dmu_f, index=model.reaction_ids, columns=["dmu/df"])
df


> dmu/df:


,dmu/df
tC,3.601237
tP,2.782637
tN,2.477095
tAce,-0.000615
Resp,5.261154
Ferm,3.569710
Nucprod,-3.325263
Aaprod,-0.736266
Replication,0.122090
Transcription,2.302624


In [29]:
print("\n> GCC_f:")
df = pd.DataFrame(data=model.GCC_f, index=model.reaction_ids, columns=["GCC_f"])
df


> GCC_f:


,GCC_f
tC,0.000000
tP,-0.818601
tN,-1.124143
tAce,-0.000615
Resp,5.261154
Ferm,3.569710
Nucprod,-3.325263
Aaprod,-0.736266
Replication,0.122090
Transcription,2.302624
